In [ ]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv


from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import AzureChatOpenAI  

dotenv.load_dotenv("~/.env")

AZURE_OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')

client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    },
)

print(client.is_ready())


In [ ]:
df = pd.read_parquet("compared.parquet")
df

In [ ]:
print(df)

In [ ]:
from weaviate.classes.query import Filter


In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for item in collection.iterator():
    print(item.uuid, item.properties)

In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for document_uuid in df.document_sha.unique():

    collection.data.delete_many(
       where=Filter.by_id().contains_any(['108bda91-73bf-55ff-9c80-510e547ea471']) 
    )

In [ ]:
client.collections.delete(name=COLLECTION_NAME)


In [2]:
import json

{'recipe_name': 'Pescado Polbano con Berros y Ejotes',
 'ingredients': "2 pizcas de sal\n2 filetes de pescado\n4 cucharadas de aceite\n2 tazas de papa, cocida, pelada y cortada en cubos medianos.\n2 tazas de ejote\n3 cucharadas de aceite de oliva\n2 pizcas de sal\n1 cucharada de mantequilla\n1 lata de crema de chile poblano, Campbell's ®\n1 taza de berro",
 'instrunctions': 'Precalienta el horno a 190°c\nUnta con sal los filetes, calienta el aceite en un sartén a fuego medio y fríe los filetes por 2 minutos. Retira del fuego y poner en una charola para horno.\nMezcla las papas con los ejotes, aceite de oliva, orégano, sal. Extiende en la misma charola donde están los filetes y mete al horno por 10 minutos.\nCalienta la mantequilla a fuego bajo en una ollita pequeña, agrega la crema de chile poblano más una lata de agua y cocina por 5 minutos para espesar un poco.\nSaca del horno la charola. Sirve unas cucharadas de crema en un plato hondo, sobre esta los vegetales, coloca el filete sob

In [20]:
import pandas as pd
import json 
from weaviate.util import 


def preprocess(path:str) -> pd.DataFrame:

    files_to_process = list_files_from_fs(f"{path}/raw")

    df = pd.DataFrame(columns=["recipe_name", "chunk"])

    for file in files_to_process:
        with open(file, 'r') as f:
            data = json.load(f)
            recipe_name = file.split("/")[-1]
        df.loc[len(df)] = {"recipe_name": recipe_name, "chunk": f"{recipe_name} \n Ingredients:{data['ingredients']}"}  
        df.loc[len(df)] = {"recipe_name": recipe_name, "chunk": f"{recipe_name} \n Instructions:{data['instructions']}"}  


    df = (
        df
        .assign(
            chunk = lambda df: df.chunk.astype(str).str.strip(),
            recipe_uuid = lambda df: [generate_uuid5(file) for file in df.recipe_name],
            chunk_uuid = lambda df: [generate_uuid5(file) for file in df.chunk],
            
        )
        .reset_index(drop=True)
    )

    save_df_in_minio(df, path, "preprocessed")


,recipe_name,chunk,recipe_uuid,chunk_uuid
0,pescado-poblano-con-berros-y-ejotes.json,pescado-poblano-con-berros-y-ejotes.json \n In...,f0b28db5-e3ea-54cd-a0e5-5ebc4a48d834,87945c8d-e089-5069-b3ad-dc545890dba3
1,pescado-poblano-con-berros-y-ejotes.json,pescado-poblano-con-berros-y-ejotes.json \n In...,f0b28db5-e3ea-54cd-a0e5-5ebc4a48d834,ba29ff15-5862-5af6-9ee4-094587811c18


In [14]:
data

{'recipe_name': 'Pescado Polbano con Berros y Ejotes',
 'ingredients': "2 pizcas de sal\n2 filetes de pescado\n4 cucharadas de aceite\n2 tazas de papa, cocida, pelada y cortada en cubos medianos.\n2 tazas de ejote\n3 cucharadas de aceite de oliva\n2 pizcas de sal\n1 cucharada de mantequilla\n1 lata de crema de chile poblano, Campbell's ®\n1 taza de berro",
 'instrunctions': 'Precalienta el horno a 190°c\nUnta con sal los filetes, calienta el aceite en un sartén a fuego medio y fríe los filetes por 2 minutos. Retira del fuego y poner en una charola para horno.\nMezcla las papas con los ejotes, aceite de oliva, orégano, sal. Extiende en la misma charola donde están los filetes y mete al horno por 10 minutos.\nCalienta la mantequilla a fuego bajo en una ollita pequeña, agrega la crema de chile poblano más una lata de agua y cocina por 5 minutos para espesar un poco.\nSaca del horno la charola. Sirve unas cucharadas de crema en un plato hondo, sobre esta los vegetales, coloca el filete sob